# RAG z wykorzystaniem Azure AI Agent Service i Semantic Kernel

Ten fragment kodu pokazuje, jak utworzyć i zarządzać agentem Azure AI do generacji wspomaganej przez wyszukiwanie (RAG) przy użyciu `Azure AI Agent Service` i `Semantic Kernel`. Agent przetwarza zapytania użytkownika na podstawie odnalezionego kontekstu i odpowiednio dostarcza precyzyjne odpowiedzi.


Naprawa wersji SQLite  
Jeśli napotkasz błąd:  
```
RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0
```  

Odkomentuj ten blok kodu na początku swojego notatnika:  


In [ ]:
# %pip install pysqlite3-binary
# __import__('pysqlite3')
# import sys
# sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

### Importowanie pakietów
Poniższy kod importuje niezbędne pakiety:


In [ ]:
# Azure imports for project client and credentials
from azure.ai.projects.models import FileSearchTool, OpenAIFile, VectorStore
from azure.identity.aio import DefaultAzureCredential

# Semantic Kernel imports
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentThread

# Generowanie wspomagane wyszukiwaniem z Semantic Kernel i Azure AI Agent Service

Ten przykład pokazuje, jak używać **Azure AI Agent Service** do przeprowadzania **Generowania wspomaganego wyszukiwaniem (RAG)** poprzez połączenie modelu językowego z kontekstem specyficznym dla danej dziedziny, pochodzącym z przesłanego dokumentu.

### Jak to działa

1. **Przesyłanie dokumentu**: Plik markdown (document.md) zawierający informacje (polisa ubezpieczeniowa podróżna firmy Contoso) jest przesyłany do usługi agenta.

2. **Tworzenie magazynu wektorowego**: Dokument jest indeksowany w magazynie wektorowym, co umożliwia semantyczne wyszukiwanie jego treści.

3. **Konfiguracja agenta**: Agent jest uruchamiany z użyciem modelu `gpt-4o` z następującymi rygorystycznymi instrukcjami:
   - Odpowiadaj tylko na pytania na podstawie odnalezionych treści z dokumentu.
   - Odmawiaj odpowiedzi, jeśli pytanie wykracza poza zakres dokumentu.

4. **Integracja narzędzia wyszukiwania plików**: `FileSearchTool` jest rejestrowane w agencie, co pozwala modelowi wyszukiwać i pobierać odpowiednie fragmenty z zaindeksowanego dokumentu podczas wnioskowania.

5. **Interakcja z użytkownikiem**: Użytkownicy mogą zadawać pytania. Jeśli w dokumencie znajdą się odpowiednie informacje, agent generuje odpowiedź opartą na danych.  
   Jeśli nie, agent wyraźnie informuje, że dokument nie zawiera wystarczających informacji.


### Główna Funkcja



Upewnij się, że najpierw uruchomisz `az login` za pomocą Azure CLI, aby zapewnić odpowiedni kontekst uwierzytelniania podczas korzystania z `DefaultAzureCredential`. Usługa Azure AI Agent Service nie używa kluczy API.


In [ ]:
async def main():
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        file: OpenAIFile = await client.agents.upload_file_and_poll(file_path="document.md", purpose="assistants")
        vector_store: VectorStore = await client.agents.create_vector_store_and_poll(
            file_ids=[file.id], name="my_vectorstore"
        )

        # Define agent name and instructions tailored for RAG.
        AGENT_NAME = "RAGAgent"
        AGENT_INSTRUCTIONS = """
        You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

        - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
        "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
        - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
        - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
        - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
        - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

        You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
        """

        
        # Create file search tool with uploaded resources
        file_search = FileSearchTool(vector_store_ids=[vector_store.id])

        # 3. Create an agent on the Azure AI agent service with the file search tool
        agent_definition = await client.agents.create_agent(
            model="gpt-4o",  # This model should match your Azure OpenAI deployment.
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            tools=file_search.definitions,
            tool_resources=file_search.resources,
        )
        
        # Create the Azure AI Agent using the client and definition.
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )
        
        # Create a thread to hold the conversation
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        # Example user queries.
        user_inputs = [
            "Can you explain Contoso's travel insurance coverage?",  # Relevant context.
            "What is a Neural Network?"  # No relevant context from the document. Will not contain a source annotation.
        ]
        
        try:
            for user_input in user_inputs:
                print(f"# User: '{user_input}'")
                # Invoke the agent for the specified thread for response
                async for response in agent.invoke(messages=user_input, thread=thread):
                    print(f"# {response.name}: {response}")
                    thread = response.thread
        finally:
            # Clean up resources.
            await thread.delete() if thread else None
            await client.agents.delete_vector_store(vector_store.id)
            await client.agents.delete_file(file.id)
            await client.agents.delete_agent(agent.id)
            print("\nCleaned up agent, thread, file, and vector store.")

await main()

Proszę podać treść pliku Markdown, który mam przetłumaczyć.



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczeniowej AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
